clone coded from : https://www.kaggle.com/code/taroz1461/carrier-smoothing-robust-wls-kalman-smoother/notebook

In [18]:
!pip install pymap3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import numpy as np
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import matplotlib.pyplot as plt
import glob as gl
import scipy.optimize
import numpy as np
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import matplotlib.pyplot as plt
import glob as gl
import scipy.optimize

CLIGHT = 299_792_458
RE_WGS84 = 6_378_137
OMGE = 7.2921151467e-5

In [20]:
def ssatellite_sellection(df, column):
  idx = df[column].notnull()
  idx &= df['CarrierErrorHz'] < 2.0e6
  idx &= df['SvElevationDegrees'] > 10.0
  idx &= df['Cn0DbHz'] > 15.0
  idx &= df['MultipathIndicator'] == 0

  return df[idx]

In [21]:
def los_vector(xusr, xsat):
  u = xsat - xusr
  rng = np.linalg.norm(u, axis=1).reshape(-1, 1)
  u /= rng
  return u, rng.reshape(-1)

In [22]:
def jac_pr_residuals(x, xsat,  pr, W):
  u, _ = los_vector(x[:3], xsat)
  J = np.hstack([-u, np.ones([len(pr), 1])])

  return W @ J

In [23]:
def pr_residuals(x, xsat, pr, W):
  u, rng = los_vector(x[:3], xsat)
  rng += OMGE * (xsat[:, 0] * x[1] - xsat[:, 1] * x[0]) / CLIGHT
  residuals = rng - (pr - x[3])

  return residuals @ W

In [24]:
def jac_prr_residuals(v, vsat, prr, x, xsat, W):
  u, _ = los_vector(x[:3], xsat)
  J = np.hstack([-u, np.ones([len(prr), 1])])

  return W @ J

In [33]:
def prr_residuals(v, vsat, prr, x, xsat, W):
  u, rng = los_vector(x[:3], xsat)
  rate = np.sum((vsat-v[:3])*u, axis=1)\
        + OMGE / CLIGHT * (vsat[:, 1] * x[0] + xsat[:, 1] * v[0]\
                           - vsat[:, 0] * x[1] - xsat[:, 0] * v[1])
  residuals = rate - (prr - v[3])

  return residuals @ W  

In [68]:
def carrier_smoothing(gnss_df):
  carr_th = 1.5
  pr_th = 20.0

  prsmooth   = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)
  for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):
    df = df.replace(
        {'AccumulatedDeltaRangeMeters': {0: np.nan}})

    drng1 = df['AccumulatedDeltaRangeMeters'].diff()\
            - df['PseudorangeRateMetersPerSecond']
    drng2 = df['RawPseudorangeMeters'].diff()\
            - df['PseudorangeRateMetersPerSecond']

    slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0
    slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0
    slip3 = np.fabs(drng1.to_numpy()) > carr_th
    slip4 = np.fabs(drng2.to_numpy()) > pr_th

    idx_slip = slip1 | slip2 | slip3 | slip4
    idx_slip[0] = True

    df['group_slip'] = np.cumsum(idx_slip)

    df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']
    meandpc = df.groupby('group_slip')['dpc'].mean()
    df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean'))

    idx = (gnss_df['Svid'] == svid_sigtype[0])\
         & (gnss_df['SignalType'] == svid_sigtype[1])

    prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']
    
  idx_nan = np.isnan(prsmooth)
  prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]
  gnss_df['pr_smooth'] = prsmooth

  return gnss_df

In [32]:
# carrier_smoothing replace function
d = pd.DataFrame(columns=['A'])
d['A'] = np.zeros(10)
d['A'] = d['A'].astype(int)
d = d.replace(
    {'A': {0:np.nan}}
)
d

,A
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [67]:
d = pd.DataFrame(columns=['A'])
d['A'] = np.arange(10)
slip = (d['A'].to_numpy() & 2)
slip

array([0, 0, 2, 2, 0, 0, 2, 2, 0, 0])